In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window
from pyspark.sql.types import StructType, StringType, IntegerType, TimestampType

# SparkSession z obsługą Kafka i okien czasowych
spark = SparkSession.builder \
    .appName("WindowedAggregation") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

# Schemat danych JSON z timestampem
schema = StructType() \
    .add("id", IntegerType()) \
    .add("name", StringType()) \
    .add("timestamp", TimestampType())

# Wczytanie danych z Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka_streaming_lab:9092") \
    .option("subscribe", "spark-lab4-topic") \
    .option("startingOffsets", "latest") \
    .load()

# Parsowanie JSON z pola value
parsed = df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

# Agregacja z użyciem okien czasowych
aggregated = parsed \
    .withWatermark("timestamp", "20 seconds") \
    .groupBy(
        window(col("timestamp"), "10 seconds")
    ) \
    .count()

# Wypisanie wyniku na konsolę
query = aggregated.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()
